<a href="https://colab.research.google.com/github/Soroushav/llm_basics/blob/main/meeting_minutes(audio_to_text).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bitsandbytes

In [ ]:
import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive, userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [ ]:
LLAMA = "meta-llama/Llama-3.1-8B-Instruct"

In [ ]:
drive.mount('/content/drive')
audio_file = '/content/drive/MyDrive/llm_basics/denver_extract.mp3'

In [ ]:
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [ ]:
hf_token = userdata.get("HF_TOKEN")
login(hf_token, add_to_git_credential=True)

In [ ]:
audio_file = open(audio_file, "rb")

In [ ]:
AUDIO_MODEL = "gpt-4o-mini-transcribe"
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

In [ ]:
display(Markdown(transcription))

In [ ]:
system_message = """
You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.
"""

user_prompt = f"""
Below is an extract transcript of a Denver council meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
{transcription}
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
cache_dir= "/content/drive/MyDrive/llm_basics/hf_cache"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA, cache_dir=cache_dir)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)

model = AutoModelForCausalLM.from_pretrained(LLAMA, cache_dir=cache_dir, quantization_config=quant_config, device_map="auto")
model.generate(inputs, max_new_tokens=2000, streamer=streamer)
